In [8]:
import easyocr
from easyocr.detection import get_detector, get_textbox
import torch   
import cv2

# Load the custom detection model
save_pth = torch.load("model/pretrain/CRAFT_clr_amp_29500.pth")
model = save_pth["craft"]
torch.save(model, "craft_detection_model.pth")

# Initialize the EasyOCR reader with the custom detection model
reader = easyocr.Reader(
    lang_list=["en"],
    detector=False,
)

reader.get_detector, reader.get_textbox = get_detector, get_textbox
reader.detector = reader.initDetector("craft_detection_model.pth")

# Read the image
img = cv2.imread("CRAFT_data/ch4_test_images/561.jpg")

reader.readtext(img)


[([[419, 151], [1026, 151], [1026, 284], [419, 284]],
  'REMA 1000',
  0.708197869881396),
 ([[487, 338], [812, 338], [812, 405], [487, 405]],
  'Salgskvittering',
  0.9999672191097156),
 ([[361, 461], [486, 461], [486, 515], [361, 515]],
  'REMA',
  0.9997764229774475),
 ([[516, 464], [621, 464], [621, 513], [516, 513]],
  '1000',
  0.9709287285804749),
 ([[651, 459], [900, 459], [900, 513], [651, 513]],
  'MAJORSTUA',
  0.9998091765601732),
 ([[421, 524], [488, 524], [488, 572], [421, 572]], 'MT', 0.9926070431703067),
 ([[532, 526], [775, 526], [775, 575], [532, 575]],
  'DAGLIGVARE',
  0.9981518857435556),
 ([[808, 529], [870, 529], [870, 578], [808, 578]], 'AS', 0.9944669040780405),
 ([[310, 581], [399, 581], [399, 629], [310, 629]], 'ORG', 0.9960464033515521),
 ([[440, 583], [518, 583], [518, 632], [440, 632]], 'NR:', 0.9144032196238746),
 ([[532, 583], [616, 583], [616, 635], [532, 635]], '260', 0.9999955263428388),
 ([[646, 581], [727, 581], [727, 629], [646, 629]], '358', 0.965

In [9]:
import easyocr
from easyocr.detection import get_detector, get_textbox
import torch   
import cv2
import numpy as np 

# Load the custom detection model
# save_pth = torch.load("model/pretrain/CRAFT_clr_amp_29500.pth")
save_pth = torch.load("./exp/custom_data_train/CRAFT_clr_amp_100.pth")
model = save_pth["craft"]
torch.save(model, "craft_detection_model.pth")

# Initialize the EasyOCR reader with the custom detection model
reader = easyocr.Reader(
    lang_list=["en"],
    detector=False,
)

reader.get_detector, reader.get_textbox = get_detector, get_textbox
reader.detector = reader.initDetector("craft_detection_model.pth")

# Read the image
img = cv2.imread("CRAFT_data/ch4_test_images/561.jpg")

# Perform text detection
results = reader.readtext(img)

# Draw bounding boxes and text on the image
for result in results:
    bbox, text, _ = result
    bbox = np.array(bbox).astype(int)
    cv2.polylines(img, [bbox], isClosed=True, color=(0, 255, 0), thickness=2)
    cv2.putText(img, text, (bbox[0][0], bbox[0][1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

# Save or display the image
output_file = "output_with_bboxes.jpg"
cv2.imwrite(output_file, img)
# Alternatively, to display the image
# cv2.imshow("Detected Text", img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

print(f"Image with bounding boxes saved to {output_file}")


Image with bounding boxes saved to output_with_bboxes.jpg


In [4]:
import easyocr
from easyocr.detection import get_detector
import torch
import cv2
import numpy as np  # Make sure numpy is imported

# Load the custom detection model
save_pth = torch.load("exp/custom_data_train/CRAFT_clr_amp_100.pth")
model = save_pth["craft"]
torch.save(model, "craft_detection_model.pth")

# Initialize the EasyOCR reader with the custom detection model
reader = easyocr.Reader(
    lang_list=["en"],
    detector=False,
)

reader.get_detector = get_detector
reader.detector = reader.initDetector("craft_detection_model.pth")

# Define the detect function based on the provided EasyOCR code
def detect(detector, img, min_size=20, text_threshold=0.7, low_text=0.4,
           link_threshold=0.4, canvas_size=2560, mag_ratio=1.0,
           slope_ths=0.1, ycenter_ths=0.5, height_ths=0.5,
           width_ths=0.5, add_margin=0.1, reformat=True, optimal_num_chars=None,
           threshold=0.2, bbox_min_score=0.2, bbox_min_size=3, max_candidates=0):

    if reformat:
        img, img_cv_grey = easyocr.utils.reformat_input(img)

    text_box_list = detector.detect(
        img, min_size=min_size,
        text_threshold=text_threshold,
        link_threshold=link_threshold,
        low_text=low_text,
        canvas_size=canvas_size,
        mag_ratio=mag_ratio,
        slope_ths=slope_ths,
        ycenter_ths=ycenter_ths,
        height_ths=height_ths,
        width_ths=width_ths,
        add_margin=add_margin,
        reformat=reformat,
        optimal_num_chars=optimal_num_chars,
        threshold=threshold,
        bbox_min_score=bbox_min_score,
        bbox_min_size=bbox_min_size,
        max_candidates=max_candidates
    )

    horizontal_list_agg, free_list_agg = [], []
    for text_box in text_box_list:
        horizontal_list, free_list = easyocr.utils.group_text_box(
            text_box, slope_ths, ycenter_ths, height_ths,
            width_ths, add_margin, (optimal_num_chars is None)
        )
        if min_size:
            horizontal_list = [i for i in horizontal_list if max(
                i[1] - i[0], i[3] - i[2]) > min_size]
            free_list = [i for i in free_list if max(
                easyocr.utils.diff([c[0] for c in i]), easyocr.utils.diff([c[1] for c in i])) > min_size]
        horizontal_list_agg.append(horizontal_list)
        free_list_agg.append(free_list)

    return horizontal_list_agg, free_list_agg

# Read the image
img = cv2.imread("CRAFT_data/ch4_test_images/561.jpg")

# Get the bounding boxes
horizontal_list_agg, free_list_agg = detect(reader.detector, img)

# Draw bounding boxes on the image
for bbox_list in horizontal_list_agg + free_list_agg:
    for bbox in bbox_list:
        pts = np.array(bbox, np.int32).reshape((-1, 1, 2))
        cv2.polylines(img, [pts], isClosed=True, color=(0, 255, 0), thickness=2)

# Save or display the image
cv2.imwrite("output_with_bboxes.jpg", img)
# Alternatively, to display the image
# cv2.imshow("Detected Text", img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


AttributeError: 'DataParallel' object has no attribute 'detect'

In [2]:
import easyocr
from easyocr.detection import get_detector
import torch
import cv2
import numpy as np

# Load the custom detection model
save_pth = torch.load("exp/custom_data_train/CRAFT_clr_amp_100.pth")
model = save_pth["craft"]
torch.save(model, "craft_detection_model.pth")

# Initialize the EasyOCR reader with the custom detection model
reader = easyocr.Reader(
    lang_list=["en"],
    detector=False,
)

# Set custom detector and load model
reader.get_detector = get_detector
reader.detector = reader.initDetector("craft_detection_model.pth")

# Define the detect function based on the provided EasyOCR code
def detect(detector, img, min_size=20, text_threshold=0.7, low_text=0.4,
           link_threshold=0.4, canvas_size=2560, mag_ratio=1.0,
           slope_ths=0.1, ycenter_ths=0.5, height_ths=0.5,
           width_ths=0.5, add_margin=0.1, reformat=True, optimal_num_chars=None,
           threshold=0.2, bbox_min_score=0.2, bbox_min_size=3, max_candidates=0):

    if reformat:
        img, img_cv_grey = easyocr.utils.reformat_input(img)

    # Perform text detection using the detector directly
    bboxes = detector.detect(
        img, min_size=min_size,
        text_threshold=text_threshold,
        link_threshold=link_threshold,
        low_text=low_text,
        canvas_size=canvas_size,
        mag_ratio=mag_ratio,
        slope_ths=slope_ths,
        ycenter_ths=ycenter_ths,
        height_ths=height_ths,
        width_ths=width_ths,
        add_margin=add_margin,
        reformat=reformat,
        optimal_num_chars=optimal_num_chars,
        threshold=threshold,
        bbox_min_score=bbox_min_score,
        bbox_min_size=bbox_min_size,
        max_candidates=max_candidates
    )

    return bboxes

# Read the image
img = cv2.imread("CRAFT_data/ch4_test_images/561.jpg")

# Get the bounding boxes
bboxes = detect(reader.detector, img)

# Draw bounding boxes on the image
for bbox in bboxes:
    pts = np.array(bbox, np.int32).reshape((-1, 1, 2))
    cv2.polylines(img, [pts], isClosed=True, color=(0, 255, 0), thickness=2)

# Save or display the image
cv2.imwrite("output_with_bboxes.jpg", img)
# Alternatively, to display the image
# cv2.imshow("Detected Text", img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


AttributeError: 'DataParallel' object has no attribute 'detect'